In [11]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."

In [ ]:
tokens = list(text.encode('utf-8'))
print(type(tokens[0]), len(tokens), tokens[:10])

<class 'int'> 616 [239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140, 240, 159, 135, 169, 226, 128, 140, 240, 159, 135, 170, 33, 32, 240, 159, 152, 132, 32, 84, 104, 101, 32, 118, 101, 114, 121, 32, 110, 97, 109, 101, 32, 115, 116, 114, 105, 107, 101, 115, 32, 102, 101, 97, 114, 32, 97, 110, 100, 32, 97, 119, 101, 32, 105, 110, 116, 111, 32, 116, 104, 101, 32, 104, 101, 97, 114, 116, 115, 32, 111, 102, 32, 112, 114, 111, 103, 114, 97, 109, 109, 101, 114, 115, 32, 119, 111, 114, 108, 100, 119, 105, 100, 101, 46, 32, 87, 101, 32, 97, 108, 108, 32, 107, 110, 111, 119, 32, 119, 101, 32, 111, 117, 103,

In [13]:
# most common pairs
def get_stats(ids):
  stats = {}
  for pair in zip(ids, ids[1:]):
    stats[pair] = stats.get(pair, 0) + 1
  return stats

In [14]:
stats = get_stats(tokens)
print(len(stats))
top_pair = max(stats, key=stats.get)
print(top_pair)

269
(101, 32)


In [20]:
# merge specific pair
def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and (ids[i], ids[i+1]) == pair:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

In [21]:
tokens2 = merge(tokens, top_pair, 256)
len(tokens2)

596

In [24]:
# train tokenizer: decide on new vocab size, iterate training dataset vocab size - 256  times and do merges for most frequent pair
vocab_size = 272
num_iters = vocab_size - 256
merges = {}
ids = list(tokens)

for i in range(num_iters):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  print(f'merging {pair} into {idx}')
  ids = merge(ids, pair, idx)
  merges[pair] = idx

merging (101, 32) into 256
merging (240, 159) into 257
merging (226, 128) into 258
merging (105, 110) into 259
merging (115, 32) into 260
merging (97, 110) into 261
merging (116, 104) into 262
merging (257, 133) into 263
merging (257, 135) into 264
merging (97, 114) into 265
merging (239, 189) into 266
merging (258, 140) into 267
merging (267, 264) into 268
merging (101, 114) into 269
merging (111, 114) into 270
merging (116, 32) into 271


In [38]:
# decode, using vocab built from merges
vocab = {i:bytes([i]) for i in range(256)}
for (p1,p2), idx in merges.items():
  vocab[idx] = vocab[p1] + vocab[p2]

def decode(ids):
  # given encoded ids, return string
  return b''.join(vocab[i] for i in ids).decode('utf-8', errors='replace')

In [ ]:
text2 = decode(ids)
text2 == text

True

In [40]:
decode([128])

'�'